# **Building a Small Language Model**


# **Stage One - Building the LLM**
This Stage Consists of 3 Parts:
1. Data Preparation and Sampling
2. Attention Mechanisms
3. LLM Architecture

## Part 1 - *Data Preparation & Sampling*
Formatting and preparing the data into the necessary form to be processed and understood by the neural network.

### Importing the Dataset Example
In our case, we will be using the pdf form of the book "The Verdict"

In [18]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x2440a2ef110>)

In [19]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Creating the Tokenizer

#### Step 1: Creating Tokens

In [20]:
# Use regular expressions to create tokens.
# We want to filter out whiite spaces and special characters.
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print(preprocessed[:50])

# Will filter out any whitespaces and only return the characters.
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:50])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ']
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']


#### Step 2: Determining the Vocabulary and mapping them to their token IDs


In [21]:
# Creating our Vocabulary of unique tokens (Alphabetically Arranged)
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

vocab = {token:integer for integer,token in enumerate(all_words)}

# Print first 50 vocab elements
for i, item in enumerate(vocab.items()):
  print(item)
  if i >= 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [22]:
vocab = {token:integer for integer, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [23]:
"""
SimpleTokenizerV1 - A simple tokenizer that can perform encoding and decoding.

SimpleTokenizerV2 - Replaces Uknown words with the special character <|unk|> and
unrelated pieces of texts with <|endoftext|>

"""
#  class SimpleTokenizerV1:
#     def __init__(self, vocab):
#       self.str_to_int = vocab
#       self.int_to_str = {i:s for s,i in vocab.items()}

#     def encode(self, text):
#       preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
#       preprocessed = [
#       item.strip() for item in preprocessed if item.strip()
#       ]
#       ids = [self.str_to_int[s] for s in preprocessed]
#       return ids

#     def decode(self, ids):
#       text = " ".join([self.int_to_str[i] for i in ids])

#       text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
#       return text

'\nSimpleTokenizerV1 - A simple tokenizer that can perform encoding and decoding.\n\nSimpleTokenizerV2 - Replaces Uknown words with the special character <|unk|> and\nunrelated pieces of texts with <|endoftext|>\n\n'

In [24]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = { i:s for s,i in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [
    item.strip() for item in preprocessed if item.strip()
    ]

    preprocessed = [item if item in self.str_to_int
                else "<|unk|>" for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
    return text

##### Create Byte-Pair Encoder
- Is a subword tokenization algorithm. The most common pair of consecutive bytes of data is replaced with a byte that does not occur in data.

Advantages:
- Byte-pair encoding can reduce the size of the vocabulary significantly.
- The BPE tokenizer can handle any unknown word without needing the `<|unk|>` token.

In [25]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [26]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text)) # Prints the new number of tokens using the GPT2 tokenizer

5145


In [27]:
# To better visualize what's being done

enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
z = enc_sample[2:context_size+2]
print(f"x: {x}")
print(f"y:      {y}")
print(f"z:            {z}")
print("------------------------------------")

# Representation: left side = input, right side = target
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(context, "---->", desired)
  print(tokenizer.decode(context), "---->", tokenizer.decode([desired])) # Text equivalent
  print("")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]
z:            [2241, 287, 257, 4489]
------------------------------------
[290] ----> 4920
 and ---->  established

[290, 4920] ----> 2241
 and established ---->  himself

[290, 4920, 2241] ----> 287
 and established himself ---->  in

[290, 4920, 2241, 287] ----> 257
 and established himself in ---->  a



In [28]:
import torch
from torch.utils.data import Dataset, DataLoader

# Dataset Implementation
class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    # Tokenizes the entire text
    token_ids = tokenizer.encode(txt)

    # Uses a sliding window approach to chunk the book into overlapping sequences.
    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i + max_length]
      target_chunk = token_ids[i + 1: i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

#-------------------------------------------------------------------------------

# Dataloader Implementation
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                 stride=128, shuffle=True, drop_last=True,
                 num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2") # Instantiates the gpt2 tokenizer

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)  # Initialize the Dataset class created earlier

      # Intantiates and provides parameters for the DataLoader python class provided by PyTorch.
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [29]:
# Test
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    dataloader = create_dataloader_v1(
      raw_text, batch_size=2, max_length=6, stride=6, shuffle=False)
    data_iter = iter(dataloader)  # Creates an Iterator
    first_batch = next(data_iter) # Gets the next batch from the data Iterator, the first_batch will be assigned a tuple of tensors.
    print(first_batch)

[tensor([[   40,   367,  2885,  1464,  1807,  3619],
        [  402,   271, 10899,  2138,   257,  7026]]), tensor([[  367,  2885,  1464,  1807,  3619,   402],
        [  271, 10899,  2138,   257,  7026, 15632]])]


In [30]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[15632,   438,  2016,   257,   922,  5891],
        [ 1576,   438,   568,   340,   373,   645]]), tensor([[ 438, 2016,  257,  922, 5891, 1576],
        [ 438,  568,  340,  373,  645, 1049]])]


#### Step 3: Mapping Token Embeddings --- COME BACK TO THIS!

##### Import Trained Model

In [31]:
#  !pip install gensim
# import gensim.downloader as api
# model = api.load("word2vec-google-news-300")

##### Positional Embedding Implementation


In [32]:
# These are the inputs that we'll be using, with their embeddings, given that each word has 3 dimensions. 
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your    
   [0.55, 0.87, 0.66], # journey 
   [0.57, 0.85, 0.64], # starts  
   [0.22, 0.58, 0.33], # with    
   [0.77, 0.25, 0.10], # one     
   [0.05, 0.80, 0.55]] # step    
)

In [34]:
"""
The Embedding function only requires the vocab_size
as it'll provide random embeddings originally
and those embeddings will be adjusted over training.
"""

vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# Instantiating the data loader
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

## Part 2 - *Attention Mechanism*
There are 4 kinds of attention mechanisms and we will be implementing each one until we arrive to the original </br>
transformer's **"Multi-Head Self-Attention"** mechanism, building on top of the previous implementation and ideas.

### First Implementation: **Simplified Attention Mechanism**

In [ ]:
query = inputs[1] # Takes 'journey' as the query.
attention_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
  attention_scores_2[i] = torch.dot(x_i, query) 

print(attention_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [37]:
attention_weights_2_tmp = attention_scores_2 / attention_scores_2.sum()

print("Attention weights: ", attention_weights_2_tmp)
print("Sum: ", attention_weights_2_tmp.sum())

Attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:  tensor(1.0000)


In [ ]:
'''
The following is a naive Softmax Implementation. However, this implementation has weaknesses when dealing with very small or large values.
Therefore it is recommended to simply use PyTorch's implmentation of Softmax.
''' 

def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attention_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())



Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [ ]:
# PyTorch Softmax Function Implementation, results in same value of previous softmax function.
attn_weights_2 = torch.softmax(attention_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


#### Calculating Context Vectors
The context vector z is the weighted sum of all input vectors, obtained by multiplying each input vector by its corresponding attention weight:

In [ ]:
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attention_weights_2_tmp[i]*x_i #x_i is simply the input vector, multiplying with its corresponding attention weights.

print(context_vec_2)

tensor([0.4355, 0.6451, 0.5680])


#### Introducing Attention Scores

In [40]:
attention_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, j_i in enumerate(inputs):
        attention_scores[i, j] = torch.dot(x_i, j_i)

print(attention_scores)

"""
The issue with the above implementation is that cause of the nested for loop
it is slow and computationally expensive, a better approach can be done with linear algebra.
"""

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


'\nThe issue with the above implementation is that cause of the nested for loop\nit is slow and computationally expensive, a better approach can be done with linear algebra.\n'

In [41]:
"""
The following method is more efficient, not because the time complexity is any less,
but the implementation time is far less, the time it takes to perform each multiplication, the '@' symbol represents matrix multiplication.
You get the exact same answer as the previous method, but much faster.
"""
attention_scores = inputs @ inputs.T
print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


#### Calculating the Attention Weights

In [42]:
attention_weights = torch.softmax(attention_scores, dim=-1) # Declaring dim =-1 will normalize accross the columns
print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


#### Computing the Context Vectors

In [43]:
all_context_vec = attention_weights @ inputs
print(all_context_vec)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


### Second Implementation: **Self-Attention**

In [44]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

NameError: name 'd_in' is not defined

In [45]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

NameError: name 'x_2' is not defined

In [46]:
attention_scores_2 = query_2 @ keys.T
attention_scores = query @ keys.T

NameError: name 'query_2' is not defined

In [ ]:
d_k = keys.shape[-1]
attention_weights_2 = torch. softmax(attention_scores_2 / d_k ** 0.5, dim=-1)
print(attention_weights_2)
print(d_k)

NameError: name 'keys' is not defined

In [ ]:
context_vec_2 = attention_weights_2 @ values
print(context_vec_2)

### Third Implementation: **Causal Attention**

### Final Implementation: **Multi-Head Attention**

# **Stage Two - Foundational Model**